# Chapter 1 : 고성능을 위한 파이썬 이해하기
**High Performance Python (고성능 파이썬)** : 파이썬 성능 잠재력을 끌어내는 실용적인 개발 전략서, 한빛 미디어


#### 이번 장에서 배울 내용
* 컴퓨터의 구성요소
* 컴퓨터 구성요소의 다양한 종류
* 파이썬에서 컴퓨터 구성요소를 추상화하는 방법
* 고성능을 위한 파이썬 코드를 작성할 때 넘어야 할 장벽
* 성능 관련 문제의 종류

---

![이미지!](img/cover_hpp.jpg)

### 들어가며 중 일부
누군가는 일련의 프로세스를 빠르게 실행해야 하고, 다른 누군가는 멀티 코어 아키텍쳐, 클러스터 또는 GPU를 제대로 활용하지 못해 애를 먹고 있을 것이다. 또 다른 누군가는 신뢰성을 잃지 않으면서도 주어진 예산 안에서 필요한 만큼의 프로세스를 사용할 수 있는 확장 가능한 시스템을 원한다. 어떤 사람은 코딩 실력의 부족함을 느끼기도 한다. 종종 다른 언어에서 차용한 기법이 예제에서 본 것만큼 자연스럽지 않은 경우도 있다.

* 컴퓨터의 저수준 동작방식을 이해하여 실제로 어떤 일이 일어나는 지 알 수 있다.
* 리스트와 튜플 : 기본적인 자료구조인 리스트와 튜플의 미묘한 의미적인 차이와 속도 차이
* 사전과 셋 : 중요한 자료구조인 사전과 셋의 메모리 할당 전략과 접근 알고리즘
* 이터레이터 : 이터레이터를 통해 데이터를 스트리밍하는 방법과 더 파이썬다운 코드를 작성하는 방법
* 순수 파이썬 방식의 접근 : 파이썬과 모듈을 효율적으로 사용하는 방법
* 행렬과 numpy : `numpy` 라이브러리를 고수처럼 사용하기
* 컴파일과 JIT : 빠른 처리를 위해 기계코드로 컴파일하기, 프로파일링 결과 반영하기
* 대이터를 효과적으로 옮기기 위한 동시적인 접근 방법
* multi-processing : 병렬 컴퓨팅을 위해 기본 라이브러리인 `multiprocessing` 모듈을 이용하는 다양한 방법, 효과적으로 `numpy` 행렬을 공유하기, 그리고 프로세스간 통신 (IPC) 비용과 장점
* 클러스터 컴퓨팅 : `multiprocessing`을 사용한 코드를 수정하여 로컬 또는 클러스터에서 실행하는 방법
* 메모리 아껴 쓰기 : 비싼 컴퓨터를 구입하지 않고 대용량 문제를 풀기 위한 접근법
* 현업에서 얻은 교훈 : 현업에서 고군분투했던 문제에서 얻은 교훈

### Python 2 vs Python 3
* 파이썬 3는 파이썬의 미래이고 모두가 파이썬 3로 갈아타는 중이다. 파이썬 2.7도 수년간 사용되긴 할 것이지만, **2020년까지만** 지원예정이다.
* 파이썬 3에서는 모든 변수가 **객체 (object)** 로 처리됩니다.
* 이외의 내부 변화가 있습니다.
    1. int 나누기 결과가 `float`
    2. `print`문 괄호 필수
    3. `str`과 `unicode`를 `str`로 통일함
    4. `long` 을 `int`로 통일
    5. `xrange` 지원하지 않는다.
    
   
    
* `__future__` 모듈

In [3]:
! git clone https://github.com/scari/high_performance_python

Cloning into 'high_performance_python'...
remote: Enumerating objects: 829, done.
remote: Total 829 (delta 0), reused 0 (delta 0), pack-reused 829
Receiving objects: 100% (829/829), 17.20 MiB | 1.19 MiB/s, done.
Resolving deltas: 100% (348/348), done.


## 1.1 컴퓨터 시스템의 기본
### 연산 장치
* IPC값이 높아지면 벡터화 수준이 증가하므로 처리 성능이 급격하게 올라간다. 벡터화란 여러개의 데이터를 입력받아 한 번에 처리하는 것을 뜻한다. 이런 종류의 CPU명령을 **SIMD (Single Instruction, multiple Data)** 라고 한다.
* **하이퍼스레딩**은 운영체제가 가상의 두 번째 CPU를 인식하게 한 다음, 똑똑한 하드웨어 로직이 단일 CPU의 실행유닛에 두 스레드를 섞어 실행하도록 하는 기법이다. 잘만 작동하면 단일 스레드 대비 30% 정도 까지 성능을 끌어올릴 수 있다. 이 기법은 두 스레드가 서로 다른 실행 유닛을 사용할 때 잘 작동한다.
* **비순차 실행**은 프로그램 실행과정에서 이전 작업 결과에 의존하지 않는 부분을 찾아내서 두 작업을 순서에 관계없이 실행하거나 혹은 동시에 실행하는 기법이다. 순서가 중요한 일만 순차적으로 처리한다면 나머지 작업이 어떤 순서로 실행되든 전체 프로그램은 정상적으로 동작하게 된다. 이 기법은 한 명령이 메모리에서 데이터를 가져오는 등의 이유로 대기하는 동안에 다른 명령을 실행할 수 있도록 함으로써 사용 가능한ㄴ 자원을 최대한 활용할 수 있게 한다.
* **멀티코어 아키텍쳐**는 하나의 실행유닛에 여러 개의 CPU를 두어 저넻적인 처리량이 단일 CPU의 처리량을 능가토록 한다. 멀티코어 아키텍쳐는 초당 처리할 수 있는 전체 연산수를 늘려주지만 모든 연산 유닛을 동시에 제대로 활용하기는 쉽지 않다.
* **암달의 법칙(Amdahl's Law)** : 멀티코어에서 작동하도록 설계된 프로그램일지라도 하나의 코어에서 실행되어야 하는 루틴이 존재하고, 이 루틴이 더 많은 코어를 투입했을 때 기대할 수 있는 최대 성능 향상치의 병목으로 작용한다는 법칙이다.
* **Global Interpretor Lock (GIL)** : 이 떄문에 여러개의 코어를 사용하기가 쉬빚않다. GIL은 코어가 몇 개든, 하나의 명령만 실행되도록 강제한다. 즉, 파이썬에서 여러개의 코어에 동시에 접근하더라도 한번에 하나의 파이썬 명령만 실행된다. `multiprocessing`, `numexpr`, `Cython`같은 기술을 이용하거나 아니면 분산 컴퓨팅 모델을 사용하는 방법으로 회피할 수 있다.

## 1.2 기본 구성 함께 보기

### 이상적인 컴퓨팅 vs 파이썬 가상 머신

In [7]:
import math

def check_prime(number):
    sqrt_number = math.sqrt(number)
    number_float = float(number)
    for i in range(2, int(sqrt_number) + 1):
        if (number_float / i).is_integer():
            return False
        return True
    
print("check_prime(10000000) =", check_prime(10000000))
print("check_prime(10000009) =", check_prime(10000009))

check_prime(10000000) = False
check_prime(10000009) = True


위 코드가 실행되면 수가 RAM에 저장된다. `sqrt_number`와 `number_float`를 계산하려면 이 값을 CPU로 보내야 한다. 이상적으로는 이 값은 한 번에 전송되어 CPU의 L1/L2 Cache에 저장되고 CPU가 계산한 결과를 RAM으로 돌려준다. FSB를 통하는 데이터 전송회수를 최소화하고 CPU와 여러 캐시를 잇는 좀더 빠른 BSB를 선택했다. 데이터를 필요한 곳에 저장하고 이동을 최소화하는 전략은 최적화에 있어서 매우 중요한 주제이다. *Heavy Data* 라는 개념은 데이터를 옮기는 데 시간과 노력이 필요하다는 사실을 의미하며 이는 피해야 한다.